# Math for ML PS 2 Programming Part
[25 points] In this programming part, you will apply eigendecomposition to implement a common dimensionality reduction and data analysis technique from machine learning: [principal components analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis). A couple of these examples are modifications of exercises from Jake VanderPlas' excellent [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html).

To submit, just fill in all the code cells with:

```
# YOUR CODE HERE #
```

You should turn in this `.ipynb` notebook into Gradescope per the homework sbumission instructions on the course webpage.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

## Eigendecomposition for Dimensionality Reduction
First, we'll consider a simple example of a synthetic dataset in $\mathbb{R}^2$. We'll first generate $n = 300$ random points, $\mathbf{x}_1, \dots, \mathbf{x}_{300} \in \mathbb{R}^2$.

In [ ]:
n = 300
d = 2

rng = np.random.RandomState(1)
X = np.dot(rng.rand(d, d), rng.randn(d, n)).T
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
print("Shape of X: {}".format(X.shape))

Although $\mathbf{X}$ is not symmetric (it's not even square, for that matter), the matrix $\mathbf{X}^{\top} \mathbf{X}$ is. The matrix $\mathbf{X}^{\top} \mathbf{X}$ is called the *covariance matrix* of the data; in a rough sense, it explains the variance of the $d = 2$ features in terms of the $n = 300$ data points we have. We do not have notions of probability in this course yet, so we will leave it at that for now.

However, we do have notions of linear algebra. We know that symmetric matrices have eigendecompositions, by the spectral theorem.

### Exercise 1 [3 points]
Write code below to find the eigendecomposition of the matrix $\mathbf{X}^{\top} \mathbf{X}$ using the function `np.linalg.eig()`. Recall that appending `.T` to a `numpy` array transposes the array. Print the resulting eigenvalues and eigenvectors.

In [ ]:
eigvals, eigvecs = np.linalg.eig(X.T @ X)

In [ ]:
# YOUR CODE HERE #

Congratulations -- you've just implemented the famous principal components analysis (PCA), which is just a synonym for "eigendecomposition of the covariance matrix." To see that, let's use `sklearn`'s official PCA implementation to check.

### Exercise 2 [3 points]
Use `sklearn` to run PCA; the [documentation is here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). To do this, set `n_components=2` and keep the rest of the hyperparameters as default. After fitting the `PCA` object, use `print(pca.components_)` and `print(pca.explained_variance_)` to print the two "principal components" of the data and the variance of the data they explain.

In [ ]:
from sklearn.decomposition import PCA

# YOUR CODE HERE #

The components that PCA returns should be *about* the same as the eigenvectors you obtained in Exercise 1 (a discrepancy in numerical precision and the algorithm that `PCA` is running under the hood accounts for this), but the eigenvalues and `explained_variance_` entries shouldn't match up. This is because the only difference is that PCA performs eigendecomposition on a slightly different matrix: $\frac{1}{n} \mathbf{X}^{\top} \mathbf{X}$. This is known as the "empirical covariance matrix," and it is still symmetric.

### Exercise 3 [3 points]
Write code below to find the eigendecomposition of the matrix $\frac{1}{n}\mathbf{X}^{\top} \mathbf{X}$ using the function `np.linalg.eig()`. Recall that appending `.T` to a `numpy` array transposes the array. Print the resulting eigenvalues and eigenvectors. Do this yourself: compare to the `components_` and `explained_variance_` values obtained from PCA.

In [ ]:
# YOUR CODE HERE #

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    color='black',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

def plot_components(X, eigvecs, eigvals):
  # plot data
  plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
  for length, vector in zip(eigvals, eigvecs):
      v = vector * 3 * np.sqrt(length)
      draw_vector(X.mean(axis=0), X.mean(axis=0) + v)
  plt.axis('equal')
  plt.show()

### Exercise 4 [3 points]
Use the function `plot_components()` supplied above to plot the eigenvectors and eigenvalues of $\frac{1}{n} \mathbf{X}^{\top} \mathbf{X}$ atop the data.

In [ ]:
# YOUR CODE HERE #

This should make clear that the famous technique of *principal components analysis (PCA)* is no more than doing eigendecomposition on the empirical covariance matrix of your collected data. The *principal components* correspond to the eigenvectors; the *explained variance* corresponds to the eigenvalues.

Now, recall that the projection operator for a 1D subspace spanned by the vector $\mathbf{u} \in \mathbb{R}^d$ is given by the $d \times d$ matrix
$$
P_{\mathbf{u}} = \frac{\mathbf{u}\mathbf{u}^{\top}}{\mathbf{u}^{\top}\mathbf{u}}.
$$
To calculate a projection for a single sample, $\mathbf{x} \in \mathbb{R}^2$, we simply compute the matrix-vector product $P_{\mathbf{u}} \mathbf{x}$. To calculate the projection for a matrix of training examples $\mathbf{X} \in \mathbb{R}^{300 \times 2}$, we need to make sure that all the *columns* of $\mathbf{X}$ include the examples, so we need to transpose $\mathbf{X}$, do a matrix-matrix multiplication, and take the transpose again: $(P_{\mathbf{u}} \mathbf{X}^{\top})^{\top} \in \mathbb{R}^{300 \times 2}$.

We'll now use the eigenvectors of $\frac{1}{n} \mathbf{X}^{\top}\mathbf{X}$ to project the matrix of examples down onto its "principal components"

### Exercise 5 [3 points]
Let the eigenvectors of $\frac{1}{n} \mathbf{X}^{\top} \mathbf{X}$ be denoted as $\mathbf{u}_1, \mathbf{u}_2$, with corresponding eigenvalues $\lambda_1 \geq \lambda_2$. Compute the $2 \times 2$ projection matrix $P_{\mathbf{u}_1}$ for the 1D subspace spanned by $\mathbf{u}_1$ and use the function `plot_project` below to plot the projection atop the original data.

In [ ]:
def plot_project(X, P_u):
  X_proj = (P_u @ X.T).T
  plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
  plt.scatter(X_proj[:, 0], X_proj[:, 1], alpha=0.8)
  plt.axis('equal')
  plt.show()

In [ ]:
# YOUR CODE HERE #

We see that we've reduced the data to the single dimensional subspace spanned by $\mathbf{u}_1$.

Let's look at another example in $\mathbb{R}^2$ just to drive the point home. We'll again generate $n = 300$ random examples in $d = 2$ dimensions. However, this time, we'll generate the data to have more equal spread in both directions.

In [ ]:
n = 300
d = 2

mu = [0, 0]
cov = np.array([[2, 0],
                [0, 2]])
X = np.random.multivariate_normal(mu, cov, size=(300,))
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
print("Shape of X: {}".format(X.shape))

### Exercise 6 [3 points]
Write code below to find the eigendecomposition of the matrix $\frac{1}{n}\mathbf{X}^{\top} \mathbf{X}$ using the function `np.linalg.eig()`. Recall that appending `.T` to a `numpy` array transposes the array. Print the resulting eigenvalues and eigenvectors.

Use the function `plot_components()` supplied above to plot the eigenvectors and eigenvalues of $\frac{1}{n} \mathbf{X}^{\top} \mathbf{X}$ atop the data.

In [ ]:
# YOUR CODE HERE #

## PCA for Eigenfaces
One domain in which we can use eigendcomposition/PCA is facial recognition. In this series of exercises, we'll use eigendecomposition to find the eigenvectors of a much more high dimensional dataset of face images. These eigenvectors will turn out to be "component faces" that we can linearly combine to construct the face images in the dataset.

We'll first load the [Labeled Faces in the Wild dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html) from `scikit-learn`. This might take a minute or so to load, depending on your internet connection.

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Let's take a look at the first four faces in the dataset.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(10, 8))
for i, ax in enumerate(axes):
  ax.set_title(faces.target_names[faces.target[i]])
  ax.imshow(faces.images[i], cmap='gray')
  ax.grid(False)

plt.show()

Notice that the above code generates $n = 1,348$ face images, but each face image is a matrix with dimensions $62 \times 47$. In order to deal with each of the faces as vectors, we'll "flatten" the face matrices into vectors so the real dataset we deal with is a matrix $\mathbf{X} \in \mathbb{R}^{1348 \times 2914}$. Here, $n = 1348$ and $d = 2914$, and we are in a very high-dimensional setting.

In [ ]:
X = faces.data
print("Shape of X: {}".format(X.shape))

### Exercise 7 [4 points]
Use `PCA` from `sklearn.decomposition` to compute the first $150$ eigenvectors (in `pca.components_`) and eigenvalues (in `pca.explained_variance_`). We wil use this instead of `np.linalg.eig` because it allows us to only compute the eigenvalues/eigenvectors up until a certain cutoff, which saves time in high-dimensional situations such as this one. In order to do this, create a `PCA()` object with `n_components=150`. Print the first $5$ eigenvectors and eigenvalues. Then, use `show_eigenfaces` with the eigenvectors you found to show the first $24$ eigenvectors and use `plot_eigvals` to plot the eigenvalues.

The [official documentation for `PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) may help.

In [ ]:
def show_eigenfaces(eigvecs):
  '''
  NOTE: expects eigvecs to be a matrix of shape (n, d), where n is the
  number of eigenvectors and d is the dimension of each eigenvector. That is,
  eigvecs should be a matrix with eigenvectors arranged row-wise.
  '''
  fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                          subplot_kw={'xticks':[], 'yticks':[]},
                          gridspec_kw=dict(hspace=0.1, wspace=0.1))
  for i, ax in enumerate(axes.flat):
      ax.imshow(eigvecs[i].reshape(62, 47), cmap='gray')
  plt.show()

def plot_eigvals(eigvals):
  '''
  NOTE: expects eigvals to be a list of eigenvalues.
  '''
  plt.plot(eigvals, 'bo')
  plt.xlabel('index of $\lambda_i$')
  plt.ylabel('value of $\lambda_i$')
  plt.show()

In [ ]:
from sklearn.decomposition import PCA
# YOUR CODE HERE #

We can see from the eigenvalue distribution in `plot_eigvals` that much of the "information" of the dataset is packed into the first $150$ eigenvectors. Recall that, for an orthonormal basis $\mathbf{u}_1, \dots, \mathbf{u}_k \in \mathbb{R}^d$ spanning a $k$-dimensional subspace $\mathcal{X}$, the projection onto that subspace is obtained by organizing the semi-orthogonal matrix $\mathbf{U} \in \mathbb{R}^{d \times k}$ with $\mathbf{u}_1, \dots, \mathbf{u}_k$ as the columns and constructing the $d \times d$ projection matrix:

$$
P_{\mathcal{X}} = \mathbf{U} \mathbf{U}^{\top}.
$$

Now, we will project the first $10$ faces onto the $k = 150$ dimension subspace. This reduces the dimensionality of the problem from $d = 2914$ to $k = 150$, which is a huge savings. We also see that this doesn't reduce the image quality too much.

### Exercise 8 [3 points]
Using the $150$ eigenvectors you found from Exercise 7, construct the projection matrix onto the $150$-dimensional subspace spanned by the eigenvectors, $P_{\mathcal{X}} = \mathbf{U} \mathbf{U}^{\top}$. Apply the projection matrix to each example in $\mathbf{X}$ (you will need to do some appropriate transposing), and use `plot_projected` to visualize the discrepancy between the original faces and the projected faces.

In [ ]:
def plot_projected(X, X_proj):
  '''
  NOTE: The argument X expects a numpy array of shape (n, d) and the argument
  X_proj expects a numpy array of shape (n, k), where k < d is the dimension of
  the projected subspace.
  '''
  # Plot the results
  k = X_proj.shape[1]
  fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                        subplot_kw={'xticks':[], 'yticks':[]},
                        gridspec_kw=dict(hspace=0.1, wspace=0.1))
  for i in range(10):
      ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
      ax[1, i].imshow(X_proj[i].reshape(62, 47), cmap='binary_r')

  ax[0, 0].set_ylabel('full-dim\ninput')
  ax[1, 0].set_ylabel('{}-dim\nprojection'.format(k))
  plt.show()

In [ ]:
# YOUR CODE HERE #

We can see that projecting these faces onto the space spanned by the first $150$ eigenvectors still leads to recognizable, albeit blurrier, results.